# Redis Assignment
In this assignment, you will access a redis server and user redis commands to find out answers.  The redis server is at *lab.aimet.tech*.  You also have to authenticate as username 'hw' with password 'hw'.  

The populated data in the redis database is similar to the example "simple social network" in the class.  Answer all questions in mycourseville assignment.

Note that this user can only use "read" commands e.g. "get", "lrange", "llen", "scan", etc.

In [1]:
# we will have to install redis in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !pip install redis

In [2]:
import redis

In [83]:
rd = redis.Redis(host='lab.aimet.tech', charset="utf-8", decode_responses=True)
rd.auth(username='hw', password='hw')

True

## What is the username of user id "600"?

In [25]:
rd.get('user:600:name')

'cautiousCrackers9'

## What is the id of username "excitedPie4" ?

In [29]:
cursor = 0
cursor, keys = rd.scan(cursor=cursor,
                       match='user:*')

while cursor > 0:
    for key in keys:
        if 'name' in key:
            name = rd.get(key)
            if name == 'excitedPie4':
                id = key.split(':')[1]
                print('id is',id)
    cursor, keys = rd.scan(cursor=cursor,
                        match='user:*')

id is 567


## How many users that "excitedPie4" follows ?

In [48]:
follower = rd.smembers('user:567:followed_by')

print(f"has {len(follower)} followers")

has 8 followers


## How many users are there in the database?

In [55]:
member_set = set()

cursor = 0
cursor, keys = rd.scan(cursor=cursor,
                       match='user:*')

while cursor > 0:
    for key in keys:
        id = key.split(':')[1]
        member_set.add(id)
    cursor, keys = rd.scan(cursor=cursor,
                        match='user:*')

print(f"total {len(member_set)} users in DB")

total 200 users in DB


## What is the average number of follows per user?

In [74]:
follows_count = dict()

cursor = 0
cursor, keys = rd.scan(cursor=cursor,
                       match='user:*')

while cursor > 0:
    for key in keys:
        id = key.split(':')[1]
        if 'follows' in key:
            print('searching id', id)
            if id not in follows_count:
                follows_set = rd.smembers(key)
                follows_count[id] = follows_set
            else:
                follows_count[id] = follows_count[id].union(follows_set)
    cursor, keys = rd.scan(cursor=cursor,
                           match='user:*')

searching id 547
searching id 569
searching id 534
searching id 611
searching id 638
searching id 669
searching id 504
searching id 592
searching id 510
searching id 555
searching id 539
searching id 617
searching id 662
searching id 516
searching id 695
searching id 640
searching id 537
searching id 546
searching id 590
searching id 654
searching id 501
searching id 614
searching id 700
searching id 530
searching id 519
searching id 571
searching id 691
searching id 604
searching id 616
searching id 521
searching id 580
searching id 559
searching id 568
searching id 646
searching id 666
searching id 587
searching id 596
searching id 593
searching id 518
searching id 606
searching id 563
searching id 686
searching id 572
searching id 542
searching id 541
searching id 623
searching id 625
searching id 668
searching id 599
searching id 503
searching id 598
searching id 557
searching id 552
searching id 641
searching id 586
searching id 520
searching id 655
searching id 658
searching id 5

In [79]:
acc_follows = 0
for key, value in follows_count.items():
    n_follows = len(value)
    acc_follows += n_follows

print(f"avg. follows is {acc_follows/200:.3}")
    

avg. follows is 8.6


## How many users follows between 5-10 users?

In [80]:
n_users = 0
for key, value in follows_count.items():
    n_follows = len(value)
    if 5 <= n_follows <= 10:
        n_users += 1

print('has', n_users, 'users')    

has 60 users


## Which account has the most followers?

In [84]:
max_followers = float('-inf')
user_id = None

cursor = 0
cursor, keys = rd.scan(cursor=cursor,
                       match='user:*')

while cursor > 0:
    for key in keys:
        id = key.split(':')[1]
        if 'followed_by' in key:
            follower = rd.smembers(key)
            if len(follower) > max_followers:
                max_followers = len(follower)
                user_id = id

    cursor, keys = rd.scan(cursor=cursor,
                           match='user:*')


In [85]:
user_name = rd.get(f"user:{id}:name")
print(f"most follower is (id={user_id}, name={user_name})")

most follower is (id=630, name=decimalGatorade1)
